In [24]:
import requests
import json
import os

from pprint import pprint
os.makedirs("data/raw", exist_ok=True)

In [2]:
def download_url(url):
    headers = {
    "accept":"application/json",
    "content-type":"application/json",
    "x-api-key":"3d996743-0e66-4c37-b4c6-20f309040d87",
    "if-none-match":"0cfeec2f23a3cff3faff5f5d08f860e42",
    "user-agent":"NTC-ios/7.9.0",
    "accept-language":"fr-fr",
    }

    resp = requests.get(url,headers=headers)
    return resp.json()

In [3]:
url = "https://standings.uefa.com/v1/standings?language=FR&competitionId=3&seasonYear=current&phase=TOURNAMENT"
groups_raw_data = download_url(url)

In [4]:
groups_raw_data[0].keys()

dict_keys(['group', 'items', 'round', 'status'])

In [3]:
def build_teams():
    
    url = "https://standings.uefa.com/v1/standings?language=FR&competitionId=3&seasonYear=current&phase=TOURNAMENT"
    groups_raw_data = download_url(url)        
    
    with open(f"data/raw/groups.json", "w") as f:
        f.write(json.dumps(groups_raw_data))

    
    teams = {}

    
    team_keys = ['associationLogoUrl','bigLogoUrl','countryCode','id','idProvider','internationalName','logoUrl','mediumLogoUrl','teamCode','teamTypeDetail',]

    for group in groups_raw_data:
        for i in group['items']:
            team_id = i['team']['id'] 
            teams[team_id] = { k:i['team'][k] for k in team_keys }
            teams[team_id]["group"]  = group["group"]["translations"]["shortName"]["EN"]
    return teams

teams = build_teams()

In [28]:
teams['43']['countryCode'] == 'FRA'

True

In [5]:
for team_id in teams:
    #print(team_id, teams[team_id]['internationalName']," ", teams[team_id]['group'],  teams[team_id]['id'], sep="\t")

66	Italy	 	A	66
128	Switzerland	 	A	128
135	Turkey	 	A	135
144	Wales	 	A	144
13	Belgium	 	B	13
35	Denmark	 	B	35
42	Finland	 	B	42
57451	Russia	 	B	57451
8	Austria	 	C	8
95	Netherlands	 	C	95
59205	North Macedonia	 	C	59205
57166	Ukraine	 	C	57166
56370	Croatia	 	D	56370
58837	Czech Republic	 	D	58837
39	England	 	D	39
117	Scotland	 	D	117
109	Poland	 	E	109
58836	Slovakia	 	E	58836
122	Spain	 	E	122
127	Sweden	 	E	127
43	France	 	F	43
47	Germany	 	F	47
57	Hungary	 	F	57
110	Portugal	 	F	110


----

In [6]:

for team_id in teams:

    print(team_id)
           
    #url = f"https://comp.uefa.com/v2/competitions/17/seasons/current/teams/{team_id}/squad?language=FR&phase=QUALIFYING"
    url = f"https://comp.uefa.com/v2/competitions/3/seasons/current/teams/{team_id}/squad?language=FR&phase=TOURNAMENT"
    composition_raw = download_url(url)
    
    os.makedirs(f"data/raw/teams/{team_id}", exist_ok=True)
    with open(f"data/raw/teams/{team_id}/composition.json", "w") as f:
        f.write(json.dumps(composition_raw))

        
    players = []

    player_keys = ["internationalName",
                    "birthDate",
                    "clubId",
                    "detailedFieldPosition",
                    "fieldPosition",
                    "id",
                    "imageUrl"]

    for players_positions in composition_raw['squad']:
        for p in players_positions['players']:

            new_player = { "jerseyNumber":p["jerseyNumber"] if "jerseyNumber" in p else None,
                          'fieldPosition' : players_positions['fieldPosition'],
                         }

            for k in player_keys:
                new_player[k] = p["player"][k] if k in p["player"] else None

            players.append(new_player)
            
    teams[team_id]['players'] = players
    
    if len(composition_raw["unlistedPlayers"]) > 0:
        print(composition_raw["unlistedPlayers"])
        
    
    coaches = []

    for c in composition_raw["coaches"]:

        new_coach = {   "imageUrl":c['imageUrl'],
                        "role":c['role'],
                        "id":c["person"]["id"],
                        "countryCode":c["person"]["countryCode"] if "countryCode" in c["person"] else None,
                        "name":c["person"]["translations"]['name']['EN'],
                    }
        coaches.append(new_coach)

    teams[team_id]['coaches'] = coaches
    

66
128
135
144
13
35
42
57451
8
95
59205
57166
56370
58837
39
117
109
58836
122
127
43
47
57
110


In [26]:
for team_id in teams:
    print(team_id, len(teams[team_id]['players']))
    #pprint(teams[team_id])

66 27
128 26
135 26
144 26
13 26
35 26
42 27
57451 26
8 26
95 26
59205 26
57166 26
56370 26
58837 26
39 26
117 26
109 26
58836 26
122 24
127 26
43 26
47 26
57 26
110 26


----

In [8]:

#player_id =  "59217" # Zlatan 
#player_id =  "93846" # Lloris

for team_id in teams:
    
    for i in range(len(teams[team_id]['players'])):

        player = teams[team_id]['players'][i]
        url = f"https://comp.uefa.com/v2/players?language=FR&playerIds={player['id']}&competitionContext=3%3Acurrent"
        player_raw = download_url(url)

        os.makedirs(f"data/raw/players/", exist_ok=True)
        with open(f"data/raw/players/{player['id']}.json", "w") as f:
            f.write(json.dumps(player_raw))

            
        teams[team_id]['players'][i]["birthDate"] = player_raw[0]['birthDate']  \
                                                                        if "birthDate" in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["clubId"] = player_raw[0]['clubId'] \
                                                                        if "clubId" in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["countryCode"] = player_raw[0]['countryCode']\
                                                                        if "countryCode" in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["detailedFieldPosition"] = player_raw[0]['detailedFieldPosition'] \
                                                                        if "detailedFieldPosition" in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["fieldPosition"] = player_raw[0]['fieldPosition']  \
                                                                        if "fieldPosition" in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["fieldPosition_FR"] = player_raw[0]['translations']['fieldPosition']['FR']
        teams[team_id]['players'][i]["height"] = player_raw[0]['height'] \
                                                                        if 'height' in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["imageUrl"] = player_raw[0]['imageUrl']  \
                                                                        if "imageUrl" in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["internationalName"] = player_raw[0]['internationalName']  \
                                                                        if "internationalName" in player_raw[0] \
                                                                        else None
        teams[team_id]['players'][i]["weight"] = player_raw[0]['weight'] \
                                                                        if 'weight' in player_raw[0] \
                                                                        else None
        
        



In [18]:
teams['43']['players'][0]

{'jerseyNumber': 16,
 'fieldPosition': 'GOALKEEPER',
 'internationalName': 'Steve Mandanda',
 'birthDate': '1985-03-28',
 'clubId': '52748',
 'detailedFieldPosition': 'GOALKEEPER',
 'id': '98893',
 'imageUrl': 'https://img.uefa.com/imgml/TP/players/3/2020/324x324/98893.jpg',
 'countryCode': 'FRA',
 'fieldPosition_FR': 'Gardien',
 'height': 185,
 'weight': 82}

----

In [19]:
clubs = {}

In [20]:
for team_id in teams:
    
    for i in range(len(teams[team_id]['players'])):

        club_id = teams[team_id]['players'][i]['clubId']
        
        if club_id is None:
            continue
        
        if club_id in clubs:
            print(clubs[club_id]['displayOfficialName'])
            continue

        url = f"https://comp.uefa.com/v2/teams?language=FR&teamIds={club_id}"
        club_raw = download_url(url)
        
        if len(club_raw) < 1:
            print(f"NO CLUB FOUND FOR ID {club_id}")
            print(url)
            continue
            
        #print(club_raw)
            
        os.makedirs(f"data/raw/clubs/", exist_ok=True)
        with open(f"data/raw/clubs/{club_raw[0]['id']}.json", "w") as f:
            f.write(json.dumps(club_raw))

        new_club = {  
            'associationLogoUrl' :   club_raw[0]['associationLogoUrl'],
            'bigLogoUrl'     :       club_raw[0]['bigLogoUrl'],
            'countryCode'    :       club_raw[0]['countryCode'],
            'idProvider'     :       club_raw[0]['idProvider'],
            'internationalName' :    club_raw[0]['internationalName'],
            'logoUrl'        :       club_raw[0]['logoUrl'],
            'mediumLogoUrl'  :       club_raw[0]['mediumLogoUrl'],
            'teamCode'       :       club_raw[0]['teamCode'],
            'teamTypeDetail' :       club_raw[0]['teamTypeDetail'],
            'displayOfficialName' :  club_raw[0]['translations']["displayOfficialName"]["EN"],
        }
        clubs[club_id] = new_club
        
        print(team_id, 
              teams[team_id]['internationalName'], 
              teams[team_id]['players'][i]['internationalName'],
              club_id, 
              new_club['displayOfficialName'],
              sep="\t")
        
                

66	Italy	Salvatore Sirigu	50029	Torino FC
66	Italy	Alex Meret	50136	SSC Napoli
66	Italy	Gianluigi Donnarumma	50058	AC Milan
66	Italy	Rafael Tolói	52816	Atalanta BC
SSC Napoli
66	Italy	Emerson	52914	Chelsea FC
66	Italy	Alessandro Florenzi	52747	Paris Saint-Germain
66	Italy	Francesco Acerbi	52973	SS Lazio
66	Italy	Leonardo Bonucci	50139	Juventus
66	Italy	Leonardo Spinazzola	50137	AS Roma
Juventus
66	Italy	Alessandro Bastoni	50138	FC Internazionale Milano
Juventus
Paris Saint-Germain
AS Roma
66	Italy	Manuel Locatelli	2600632	US Sassuolo Calcio
Chelsea FC
FC Internazionale Milano
Juventus
AS Roma
FC Internazionale Milano
Atalanta BC
Torino FC
SS Lazio
US Sassuolo Calcio
US Sassuolo Calcio
SSC Napoli
128	Switzerland	Jonas Omlin	50128	Montpellier Hérault SC
128	Switzerland	Yann Sommer	52757	VfL Borussia Mönchengladbach
128	Switzerland	Yvon Mvogo	50062	PSV Eindhoven
128	Switzerland	Manuel Akanji	52758	Borussia Dortmund
128	Switzerland	Kevin Mbabu	64332	VfL Wolfsburg
Torino FC
128	Switzerland	

In [21]:
for team_id in teams:
    
    for i in range(len(teams[team_id]['coaches'])):

        coach = teams[team_id]['coaches'][i]
        print(teams[team_id]['internationalName'], coach["id"], coach['name'])

        url = f"https://comp.uefa.com/v2/players?language=FR&playerIds={coach['id']}&competitionContext=3%3Acurrent"
        coach_raw = download_url(url)

        os.makedirs(f"data/raw/coaches/", exist_ok=True)
        with open(f"data/raw/coaches/{coach['id']}.json", "w") as f:
            f.write(json.dumps(coach_raw))

        if len(coach_raw) < 1:
            #print("coach not found")
            continue
        
        
        teams[team_id]['coaches'][i]["birthDate"] = coach_raw[0]["birthDate"]

        
        teams[team_id]['coaches'][i]["height"] = coach_raw[0]["height"] if "height" in coach_raw[0] else None
        

Italy 105 Roberto Mancini
Italy 422 Alberico Evani
Italy 823 Fausto Salsano
Italy 5634 Attilio Lombardo
Italy 58656 Daniele De Rossi
Switzerland 22672 Antonio Manicone
Switzerland 279571 Vladimir Petković
Turkey 8950 Şenol Güneş
Turkey 39084 Bayram Bektas
Turkey 250070477 Kerem Yavaş
Turkey 250073120 Seref Çiçek
Wales 8614 Rob Page
Wales 18144 Christopher Symons
Wales 108921 Albert Stuivenberg
Belgium 24057 Thierry Henry
Belgium 55522 Shaun Richard Maloney
Belgium 250016130 Roberto Martínez
Denmark 11067 Morten Wieghorst
Denmark 1903534 Kasper Hjulmand
Finland 827 Kari Martonen
Finland 3619 Mika Nurmela
Finland 287526 Toni Korkeakunnas
Finland 800344 Markku Kanerva
Russia 29516 Stanislav Cherchesov
Russia 250073752 Miroslav Romashchenko
Austria 1291 Thomas Kristl
Austria 11745 Franco Foda
Austria 57063 Jürgen Säumel
Netherlands 4761 Frank de Boer
Netherlands 9397 Dwight Lodeweges
Netherlands 38901 Rutgerus Johannes Martinus Van Nistelrooij
North Macedonia 30379 Igor Angelovski
North Ma

In [22]:
with open("data/teams.json", "w") as f:
    f.write(json.dumps(teams))

with open("data/clubs.json", "w") as f:
    f.write(json.dumps(clubs))
